In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import sqlite3

# Read the CSV file
data = pd.read_csv('../data/Food_Delivery_Times.csv')

# Examine the data structure
print("CSV columns:", data.columns.tolist())
print("\nSample data:")
print(data.head())

# Create SQLite database connection
conn = sqlite3.connect(':memory:')  # In-memory database for demonstration
# Alternatively: conn = sqlite3.connect('food_delivery.db')

# Process and create deliveries table
# Extract relevant columns if they exist or generate if needed
deliveries = pd.DataFrame()

# Map columns from CSV to our schema (adjust based on actual CSV structure)
if 'Delivery_ID' in data.columns:
    deliveries['delivery_id'] = data['Delivery_ID']
else:
    deliveries['delivery_id'] = [f'DEL{i:05d}' for i in range(1, len(data) + 1)]

# Extract or generate other columns similarly
if 'Delivery_person_ID' in data.columns:
    deliveries['delivery_person_id'] = data['Delivery_person_ID']
else:
    deliveries['delivery_person_id'] = np.random.randint(1, 101, size=len(data))

# Extract restaurant and customer area
for col_name, target_col in [('Restaurant_Area', 'restaurant_area'), 
                             ('Customer_Area', 'customer_area')]:
    if col_name in data.columns:
        deliveries[target_col] = data[col_name]
    else:
        areas = ['Downtown', 'Uptown', 'Suburb', 'Business District', 'Residential']
        deliveries[target_col] = np.random.choice(areas, size=len(data))

# Process remaining delivery columns
if 'Delivery_distance' in data.columns:
    deliveries['delivery_distance_km'] = data['Delivery_distance']
else:
    deliveries['delivery_distance_km'] = np.random.uniform(0.5, 15.0, size=len(data)).round(2)

if 'Time_taken(min)' in data.columns:
    deliveries['delivery_time_min'] = data['Time_taken(min)']
else:
    deliveries['delivery_time_min'] = np.random.randint(10, 60, size=len(data))

# Generate timestamp for order_placed_at
if 'Order_Date' in data.columns:
    deliveries['order_placed_at'] = pd.to_datetime(data['Order_Date'])
else:
    base_date = datetime(2023, 1, 1)
    deliveries['order_placed_at'] = [
        base_date + timedelta(days=random.randint(0, 365), 
                              hours=random.randint(0, 23),
                              minutes=random.randint(0, 59))
        for _ in range(len(data))
    ]

# Weather and traffic conditions
for col_name, target_col, choices in [
    ('Weather_conditions', 'weather_condition', ['Sunny', 'Rainy', 'Cloudy', 'Foggy', 'Windy']),
    ('Traffic_conditions', 'traffic_condition', ['Low', 'Medium', 'High', 'Very High'])
]:
    if col_name in data.columns:
        deliveries[target_col] = data[col_name]
    else:
        deliveries[target_col] = np.random.choice(choices, size=len(data))

if 'Rating' in data.columns:
    deliveries['delivery_rating'] = data['Rating']
else:
    deliveries['delivery_rating'] = np.random.uniform(1.0, 5.0, size=len(data)).round(1)

# Create delivery_persons table
unique_delivery_persons = pd.DataFrame({
    'delivery_person_id': deliveries['delivery_person_id'].unique()
})
delivery_persons = pd.DataFrame({
    'delivery_person_id': unique_delivery_persons['delivery_person_id'],
    'name': [f'Delivery Person {id}' for id in unique_delivery_persons['delivery_person_id']],
    'region': np.random.choice(['North', 'South', 'East', 'West'], size=len(unique_delivery_persons)),
    'hired_date': pd.date_range(start='2022-01-01', periods=len(unique_delivery_persons), freq='D').date,
    'is_active': np.random.choice([True, False], size=len(unique_delivery_persons), p=[0.9, 0.1])
})

# Create restaurants table
areas = deliveries['restaurant_area'].unique()
n_restaurants = len(areas) * 3  # 3 restaurants per area

restaurants = pd.DataFrame({
    'restaurant_id': [f'REST{i:03d}' for i in range(1, n_restaurants + 1)],
    'area': np.repeat(areas, 3),
    'name': [f'Restaurant {i}' for i in range(1, n_restaurants + 1)],
    'cuisine_type': np.random.choice(['Italian', 'Chinese', 'Indian', 'Mexican', 'American'], size=n_restaurants),
    'avg_preparation_time_min': np.random.uniform(10, 30, size=n_restaurants).round(1)
})

# Create orders table
orders = pd.DataFrame({
    'order_id': range(1, len(deliveries) + 1),
    'delivery_id': deliveries['delivery_id'],
    'restaurant_id': np.random.choice(restaurants['restaurant_id'], size=len(deliveries)),
    'customer_id': [f'CUST{i:04d}' for i in range(1, len(deliveries) + 1)],
    'order_value': np.random.uniform(10, 100, size=len(deliveries)).round(2),
    'items_count': np.random.randint(1, 10, size=len(deliveries))
})

# Save tables to database
# deliveries.to_sql('deliveries', conn, index=False, if_exists='replace')
# delivery_persons.to_sql('delivery_persons', conn, index=False, if_exists='replace')
# restaurants.to_sql('restaurants', conn, index=False, if_exists='replace')
# orders.to_sql('orders', conn, index=False, if_exists='replace')

CSV columns: ['Order_ID', 'Distance_km', 'Weather', 'Traffic_Level', 'Time_of_Day', 'Vehicle_Type', 'Preparation_Time_min', 'Courier_Experience_yrs', 'Delivery_Time_min']

Sample data:
   Order_ID  Distance_km Weather Traffic_Level Time_of_Day Vehicle_Type  \
0       522         7.93   Windy           Low   Afternoon      Scooter   
1       738        16.42   Clear        Medium     Evening         Bike   
2       741         9.52   Foggy           Low       Night      Scooter   
3       661         7.44   Rainy        Medium   Afternoon      Scooter   
4       412        19.03   Clear           Low     Morning         Bike   

   Preparation_Time_min  Courier_Experience_yrs  Delivery_Time_min  
0                    12                     1.0                 43  
1                    20                     2.0                 84  
2                    28                     1.0                 59  
3                     5                     1.0                 37  
4                  

In [ ]:
# Create a connection to the file database
file_conn = sqlite3.connect('../data/food_delivery.db')

# Copy all tables from in-memory database to file database
for table_name in ['deliveries', 'delivery_persons', 'restaurants', 'orders']:
    # Read the table from in-memory database
    df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
    
    # Write the table to file database
    df.to_sql(table_name, file_conn, index=False, if_exists='replace')

# Close both connections
conn.close()
file_conn.close()

Database successfully saved to '../data/food_delivery.db'!
Tables can now be accessed from other Python scripts.
